<a href="https://colab.research.google.com/github/Seogury/cp2_team3/blob/main/CB_and_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###CB 알고리즘(아이템 기반)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

product = pd.read_csv('/content/gdrive/MyDrive/product.csv')
product_vect = pd.read_csv('/content/gdrive/MyDrive/product_vect.csv')

In [ ]:
#벡터화
counter = CountVectorizer()
vect = counter.fit_transform(product_vect['category_brand'])
vect

<12174x1215 sparse matrix of type '<class 'numpy.int64'>'
	with 43901 stored elements in Compressed Sparse Row format>

In [ ]:
#코사인 유사도(item)
cos = cosine_similarity(vect, vect)

#데이터 프레임화
item_sim_df = pd.DataFrame(cos, index = product['product_id'], columns = product['product_id'])

In [ ]:
cos.shape

(12174, 12174)

In [ ]:
item_sim_df.to_csv('item_sim.csv')

### 코사인 유사도 데이터

In [ ]:
item_sim_df = pd.read_csv('/content/gdrive/MyDrive/item_sim.csv')

In [ ]:
item_sim_df.set_index('product_id', inplace = True)

In [ ]:
item_sim_df

In [ ]:
product

In [ ]:
def recommend_item(id, top = 11):
  #product_id 검색
  product.reset_index(inplace = True) 
  target = item_sim_df[f'{id}'].sort_values(ascending = False)[:top].index
  target_list = []
  for i in target:
      target_list.append(i)
  target_list.remove(id)
  
  #id리스트로 index 검색
  product.set_index('product_id',inplace = True)
  result = product.loc[target_list].sort_values('product_pur_count', ascending = False)
   

  return result[['brand','price','main_category','middle_category','sub_category']]

In [ ]:
recommend_item(1002098)

###cf 알고리즘


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import random
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
df3.to_csv('df3.csv')

In [ ]:
df2

In [ ]:
df3 = pd.read_csv('/content/gdrive/MyDrive/df3.csv')

In [ ]:
df_ele = df3[(df3['main_category']=='electronics')]
df_app = df3[(df3['main_category']=='appliances')]
df_com = df3[(df3['main_category']=='computers')]

In [ ]:
rate = df3.groupby(['user_id', 'product_id']).sum()['event_type'].reset_index()

In [ ]:
ele_rate = df_ele.groupby(['user_id', 'product_id']).sum()['event_type'].reset_index()
app_rate = df_app.groupby(['user_id', 'product_id']).sum()['event_type'].reset_index()
com_rate = df_com.groupby(['user_id', 'product_id']).sum()['event_type'].reset_index()

In [ ]:
app_rate_ran = app_rate.sample(frac=0.5, replace=True)

In [ ]:
len(rate['product_id'].unique())

12174

In [ ]:
len(rate[:381946]['product_id'].unique()), len(app_rate['product_id'].unique()), len(com_rate['product_id'].unique())

(12174, 4838, 2638)

In [ ]:
len(rate['product_id'].unique())

12174

In [ ]:
def item_cosine(df):

  #정규화
  mean = np.mean(df['event_type'])
  max = np.max(df['event_type'])
  min = np.min(df['event_type'])

  df['event_type'] = df.event_type.apply(lambda x: (x-mean)/(max-min))
  
  #피벗테이블
  piv = df.pivot_table(index=['user_id'], columns=['product_id'], values='event_type')
  piv.fillna(0,inplace = True)
  
  #압축
  piv_sparse = sp.sparse.csr_matrix(piv.values)

  #유사도 비교
  item_similarity = cosine_similarity(piv_sparse.T)
  
  #데이터 프레임
  item_sim_df = pd.DataFrame(item_similarity, index = piv.columns, columns = piv.columns)

  return item_sim_df

In [ ]:
def user_cosine(df):

  #정규화
  mean = np.mean(df['event_type'])
  max = np.max(df['event_type'])
  min = np.min(df['event_type'])

  df['event_type'] = df.event_type.apply(lambda x: (x-mean)/(max-min))
  
  #피벗테이블
  piv = df.pivot_table(index=['user_id'], columns=['product_id'], values='event_type')
  piv.fillna(0,inplace = True)

  piv_sparse = sp.sparse.csr_matrix(piv.values)
  user_similarity = cosine_similarity(piv_sparse)

  user_sim_df = pd.DataFrame(user_similarity, index = piv.index, columns = piv.index)

  return user_sim_df

In [ ]:
rate.info()

In [ ]:
cos_item = item_cosine(rate)

In [ ]:
com_item = item_cosine(com_rate)

In [ ]:
com_user = user_cosine(com_rate)

In [ ]:
com_user.info()

In [ ]:
com_user.to_csv('com_user.csv')

In [ ]:
com_item.to_csv('com_item.csv')
# com_user.to_csv('com_user.csv')

In [ ]:
com_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2638 entries, 1303049 to 45500030
Columns: 2638 entries, 1303049 to 45500030
dtypes: float64(2638)
memory usage: 53.2 MB


In [ ]:
app_item = item_cosine(app_rate)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
app_user = user_cosine(app_rate_ran)

In [ ]:
app_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21976 entries, 384989212 to 566236063
Columns: 21976 entries, 384989212 to 566236063
dtypes: float64(21976)
memory usage: 3.6 GB


In [ ]:
app_item.to_csv('app_item.csv')

In [ ]:
app_user.to_csv('app_user.csv')#3gb


In [ ]:
ele_item = item_cosine(ele_rate[:1000000])

In [ ]:
# ele_user = user_cosine(ele_rate)

In [ ]:
ele_user.to_csv('ele_user.csv')

In [ ]:
ele_item.to_csv('ele_item.csv')

In [ ]:
ele_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4698 entries, 1000978 to 51600020
Columns: 4698 entries, 1000978 to 51600020
dtypes: float64(4698)
memory usage: 168.6 MB


####top10

In [ ]:
ele_item

In [ ]:
def item_recommend_CF1(df, pro, id, top = 11):
  #product_id 검색
  pro.reset_index(inplace = True) 
  target = df[f'{id}'].sort_values(ascending = False)[:top].index
  target_list = []
  for i in target:
      target_list.append(i)
  target_list.remove(id)
  
  #id리스트로 index 검색
  pro.set_index('product_id',inplace = True)
  result = product.loc[target_list].sort_values('product_pur_count', ascending = False)
   

  return result[['brand','price','main_category','middle_category','sub_category']]

In [ ]:
item_recommend_CF1(ele_item, product,1002062, top = 50)